In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from dgl.nn import EdgeWeightNorm, GraphConv,RelGraphConv, SAGEConv
from dgl.data import DGLDataset
import mysql.connector
import dgl.data
import numpy as np
import pandas as pd
import os
from dgl import save_graphs, load_graphs
import matplotlib.pyplot as plt
import random
from Datasets import PaperDataset
from Models import GraphClassificationModelCrossEntropy, GraphClassificationModelBinaryCrossEntropy
import networkx as nx
import scipy.sparse as sp
import itertools
import torch as th
import dgl.function as fn
from sklearn.metrics import roc_auc_score
import dgl.nn as dglnn
import random
from torchmetrics.classification import BinaryAUROC
from torch.utils.data import TensorDataset, DataLoader

In [2]:
cnx = mysql.connector.connect(user='david', password='daviddung1993',
                          host='127.0.0.1',
                          database='computervision')
cursor = cnx.cursor()
headers = {"x-api-key": "M7HSjQNeTfai6l7JUiDZB8XYc85BHnHt3R0NXSEd"}

In [3]:
test_papers = []
with open(r'./test_papers.txt', 'r') as fp:
    for line in fp:
        x = line[:-1]
        test_papers.append(x)

train_papers = []
with open(r'./train_papers.txt', 'r') as fp:
    for line in fp:
        x = line[:-1]
        train_papers.append(x)

total_papers = train_papers + test_papers
in_params_train = ','.join(['%s'] * len(train_papers+test_papers))

val_papers = []
with open(r'./val_papers.txt', 'r') as fp:
    for line in fp:
        x = line[:-1]
        val_papers.append(x)
in_params_val = ','.join(['%s'] * len(val_papers))

In [ ]:
def remap(x, mapping):
    return mapping.get(x, x)

cursor.execute("select * from referencedBy b where b.ReferenceID in (select PaperID from Papers) and b.ReferencedByID in (select PaperID from Papers)")
paper_edges = np.array(cursor.fetchall())
all_papers = np.unique(paper_edges)
remapped_papers = {all_papers[i]: i  for i in range(len(all_papers))}
vec_remap = np.vectorize(remap, otypes=[str])
paper_edges = vec_remap(paper_edges, remapped_papers)

cursor.execute("select * from authoredBy where PaperID in (select PaperID from Papers) and (PaperID in (select ReferenceID from referencedBy) or PaperID in (select ReferencedByID from referencedBy))")
author_edges = np.array(cursor.fetchall())
all_authors = np.unique(author_edges[:,1])
remapped_authors = {all_authors[i]: i  for i in range(len(all_authors))}

author_edges[:,1] = vec_remap(author_edges[:,1], remapped_authors)
author_edges[:,0] = vec_remap(author_edges[:,0], remapped_papers)

In [5]:
cursor.execute("select distinct b.AuthoredByID from Papers p, authoredBy b where p.PaperID in (%s) and p.PaperID = b.PaperID" % in_params_train, total_papers)
train_author_ids = [x[0] for x in cursor.fetchall()]
len(train_author_ids)

6424

In [6]:
node_types = ["paper", "author"]
edge_types = ["cited", "authored"]

hetero_graph = dgl.heterograph(
    {
        ('paper', 'authored', 'author'): (author_edges[:,0].astype(int), author_edges[:,1].astype(int)),
        ('paper', 'cites', 'paper'): (paper_edges[:,0].astype(int), paper_edges[:,1].astype(int))
    }
)
hetero_graph.nodes['paper'].data['feature'] = th.rand(hetero_graph.number_of_nodes('paper'), 1)
hetero_graph.nodes['author'].data['feature'] = th.rand(hetero_graph.number_of_nodes('author'), 1)

In [8]:
def cartesian_product(A, B):
    for a in A:
        for b in B:
            yield (a, b)

def construct_negative_graph(graph, k, etype):
    utype, _, vtype = etype
    src, dst = graph.edges(etype=etype)
    neg_src = src.repeat_interleave(k)
    neg_dst = torch.randint(0, graph.num_nodes(vtype), (len(src) * k,))
    return dgl.heterograph(
        {etype: (neg_src, neg_dst)},
        num_nodes_dict={ntype: graph.num_nodes(ntype) for ntype in graph.ntypes})

def construct_negative_edge(graph, paper, etype):
    utype, _, vtype = etype
    dst = graph.predecessors(paper, etype=etype)
    neg_src = paper.repeat_interleave(graph.num_nodes(vtype) - len(dst))

    neg_mask = ~torch.isin(graph.nodes(vtype), dst)
    neg_dst = torch.masked_select(graph.nodes(vtype), neg_mask)
    return dgl.heterograph(
        {etype: (neg_src, neg_dst)},
        num_nodes_dict={ntype: graph.num_nodes(ntype) for ntype in graph.ntypes})

def construct_negative_graph_v2(graph, k, etype):
    utype, edge_type, vtype = etype
    src, dst = graph.edges(etype=etype)
    eids = graph.edge_ids(src, dst, etype=edge_type)

    neg_sampler = dgl.dataloading.negative_sampler.PerSourceUniform(k)
    neg_src, neg_dst = neg_sampler(graph, {edge_type: eids})[etype]
    return dgl.heterograph(
        {etype: (neg_src, neg_dst)},
        num_nodes_dict={ntype: graph.num_nodes(ntype) for ntype in graph.ntypes})

def construct_complete_negative_graph(graph, etype):
    utype, edge_type, vtype = etype
    src, dst = graph.edges(etype=etype)
    all_src_nodes = torch.unique(src)
    all_author_nodes = graph.nodes(vtype)
    neg_graph = dgl.heterograph({etype: ([], [])}, {ntype: graph.num_nodes(ntype) for ntype in graph.ntypes})
    src_dst_pairs = list(zip(src.tolist(), dst.tolist()))
    dataset = TensorDataset(all_author_nodes)
    dataloader = DataLoader(dataset, batch_size=2500, shuffle=False)

    for batch_idx, author_batch in enumerate(dataloader):
        print(f"{batch_idx}/{len(dataloader)}")
        complete_edges_batch = torch.cartesian_prod(all_src_nodes, author_batch[0])
        complete_edges_tuples = [(complete_edges_batch[i][0].item(), complete_edges_batch[i][1].item()) for i in range(complete_edges_batch.shape[0])]
        filtered_mask = ~torch.tensor([True if tuple in src_dst_pairs else False for tuple in complete_edges_tuples])

        filtered_edges = complete_edges_batch[filtered_mask]
        neg_src = filtered_edges[:,0]
        neg_dst = filtered_edges[:,1]
        neg_graph.add_edges(neg_src, neg_dst, etype='authored')

    return neg_graph

In [10]:
class RGCN(nn.Module):
    def __init__(self, in_feats, hid_feats, out_feats, rel_names):

        super().__init__()
        self.conv1 = dglnn.HeteroGraphConv({
            rel: dglnn.GraphConv(in_feats, hid_feats)
            for rel in rel_names}, aggregate='max')
        self.conv2 = dglnn.HeteroGraphConv({
            rel: dglnn.GraphConv(hid_feats, out_feats)
            for rel in rel_names}, aggregate='max')

    def forward(self, graph, inputs):
        h = self.conv1(graph, inputs)
        h = {k: F.relu(v) for k, v in h.items()}
        h = self.conv2(graph, h)
        return h

class HeteroDotProductPredictor(nn.Module):
    def forward(self, graph, h, etype):
        with graph.local_scope():
            graph.ndata['h'] = h
            graph.apply_edges(fn.u_dot_v('h', 'h', 'score'), etype=etype)
            return graph.edges[etype].data['score']

class Model(nn.Module):
    def __init__(self, in_features, hidden_features, out_features, rel_names):
        super().__init__()
        # Encoder
        self.sage = RGCN(in_features, hidden_features, out_features, rel_names)
        # Decoder
        self.pred = HeteroDotProductPredictor()
    def forward(self, g, neg_g, x, etype):
        h = self.sage(g, x)
        return self.pred(g, h, etype), self.pred(neg_g, h, etype)


# Metrics

def compute_auc(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score]).numpy()
    labels = torch.cat(
        [torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])]
    ).numpy()
    return roc_auc_score(labels, scores)

def compute_loss(pos_score, neg_score):
    # Margin loss
    n_edges = pos_score.shape[0]
    #print(neg_score.view(n_edges, -1).shape)
    return (1 - pos_score + neg_score.view(n_edges, -1)).clamp(min=0).mean()

def compute_loss_asymmetric(pos_score, neg_score, pos_mapping, neg_mapping):
    # Margin loss
    #n_edges = pos_score.shape[0]
    total_list = []

    for node in torch.unique(pos_mapping):
        pos_node_indices = (pos_mapping == node).nonzero(as_tuple=True)[0]
        neg_node_indices = (neg_mapping == node).nonzero(as_tuple=True)[0]

        pos_edges_score = torch.index_select(pos_score, 0, pos_node_indices).mean()
        neg_edges = torch.index_select(neg_score, 0, neg_node_indices)

        node_score = (1-pos_edges_score+neg_edges).clamp(min=0).mean()
        total_list.append(node_score)

    total_loss = torch.tensor(total_list).mean()
    return total_loss
    #return (1 - pos_score + neg_score.view(n_edges, -1)).clamp(min=0).mean()

def compute_loss_asymmetric_edges(pos_score, neg_score, pos_mapping, neg_mapping):
    total_list = []

    for idx, node in enumerate(pos_mapping):
        #pos_node_indices = (pos_mapping == node).nonzero(as_tuple=True)[0]
        neg_node_indices = (neg_mapping == node).nonzero(as_tuple=True)[0]

        neg_edges = torch.index_select(neg_score, 0, neg_node_indices)
        current_pos_score = pos_score[idx]

        node_score = (1-current_pos_score+neg_edges).clamp(min=0).mean()
        total_list.append(node_score)

    total_loss = torch.tensor(total_list).mean()
    return total_loss

In [11]:
remapped_test_papers = th.tensor([remapped_papers[idx] for idx in test_papers])
remapped_val_papers = th.tensor([remapped_papers[idx] for idx in val_papers])
remapped_train_test_author_ids = th.tensor([remapped_authors[idx] for idx in train_author_ids])

In [12]:
# Remove all Test Papers
train_hetero_graph = dgl.remove_nodes(hetero_graph, remapped_test_papers, ntype="paper")

In [15]:
val_test_hetero_graph = dgl.heterograph({
    etype: hetero_graph.edges(etype=etype)
    for etype in hetero_graph.canonical_etypes
}, num_nodes_dict={
    ntype: hetero_graph.number_of_nodes(ntype=ntype)
    for ntype in hetero_graph.ntypes
})

val_test_hetero_graph.nodes['author'].data['feature'] = hetero_graph.nodes['author'].data['feature']
val_test_hetero_graph.nodes['paper'].data['feature'] = hetero_graph.nodes['paper'].data['feature']

In [16]:
author_mask = ~torch.isin(val_test_hetero_graph.nodes("author"),remapped_train_test_author_ids)
not_author_in_question = torch.masked_select(val_test_hetero_graph.nodes("author"), author_mask)
val_positive_hetero_graph = dgl.remove_nodes(val_test_hetero_graph, not_author_in_question, ntype="author")

u, v = val_positive_hetero_graph["authored"].edges()
mask = ~torch.isin(u, remapped_val_papers)
not_pos_val_u, not_pos_val_v = torch.masked_select(u, mask), torch.masked_select(v, mask)
not_in_val_pos_ids = val_positive_hetero_graph.edge_ids(not_pos_val_u, not_pos_val_v, etype="authored")
val_positive_hetero_graph.remove_edges(not_in_val_pos_ids, etype="authored")

In [17]:
#a = val_positive_hetero_graph.edges(etype="authored")
val_negative_hetero_graph = construct_complete_negative_graph(val_positive_hetero_graph, ('paper', 'authored', 'author'))

0/3
1/3
2/3


In [ ]:
model = Model(1, 40, 40, train_hetero_graph.etypes)
author_feats = train_hetero_graph.nodes['author'].data['feature']
paper_feats = train_hetero_graph.nodes['paper'].data['feature']
node_features = {'author': author_feats, 'paper': paper_feats}
opt = torch.optim.Adam(model.parameters())
k = 5